### Set Up Packages and Global Settings

In [2]:
import fastf1 as ff1
import pandas as pd

pd.set_option('display.max_columns', None)

### Explore Schedule

In [19]:
schedule = ff1.get_event_schedule(1994)
schedule.RoundNumber.to_list()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

### Explore Session Object

In [25]:
session_quali = ff1.get_session(2000, 1, 'Q')

session_quali.load()
session_quali.results

core           INFO 	Loading data for Australian Grand Prix - Qualifying [v3.1.6]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
logger      WARNING 	Failed to load session info data!
core        WARNING 	Cannot load laps, telemetry, weather, and message data because the relevant API is not supported for this session.
core           INFO 	Finished loading data for 22 drivers: ['1', '2', '3', '4', '5', '6', '7', '22', '11', '17', '9', '18', '19', '12', '15', '23', '14', '20', '16', '8', '10', '21']


,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points
DriverNumber,,,,,,,,,,,,,,,,,,,,,
1,1,,nan,hakkinen,McLaren,,mclaren,Mika,Häkkinen,Mika Häkkinen,,,1.0,,NaN,0 days 00:01:30.556000,NaT,NaT,NaT,,NaN
2,2,,COU,coulthard,McLaren,,mclaren,David,Coulthard,David Coulthard,,,2.0,,NaN,0 days 00:01:30.910000,NaT,NaT,NaT,,NaN
3,3,,MSC,michael_schumacher,Ferrari,,ferrari,Michael,Schumacher,Michael Schumacher,,,3.0,,NaN,0 days 00:01:31.075000,NaT,NaT,NaT,,NaN
4,4,,BAR,barrichello,Ferrari,,ferrari,Rubens,Barrichello,Rubens Barrichello,,,4.0,,NaN,0 days 00:01:31.102000,NaT,NaT,NaT,,NaN
5,5,,nan,frentzen,Jordan,,jordan,Heinz-Harald,Frentzen,Heinz-Harald Frentzen,,,5.0,,NaN,0 days 00:01:31.359000,NaT,NaT,NaT,,NaN
6,6,,TRU,trulli,Jordan,,jordan,Jarno,Trulli,Jarno Trulli,,,6.0,,NaN,0 days 00:01:31.504000,NaT,NaT,NaT,,NaN
7,7,,nan,irvine,Jaguar,,jaguar,Eddie,Irvine,Eddie Irvine,,,7.0,,NaN,0 days 00:01:31.514000,NaT,NaT,NaT,,NaN
22,22,,VIL,villeneuve,BAR,,bar,Jacques,Villeneuve,Jacques Villeneuve,,,8.0,,NaN,0 days 00:01:31.968000,NaT,NaT,NaT,,NaN
11,11,,FIS,fisichella,Benetton,,benetton,Giancarlo,Fisichella,Giancarlo Fisichella,,,9.0,,NaN,0 days 00:01:31.992000,NaT,NaT,NaT,,NaN


In [16]:
session_quali = ff1.get_session(2023, 1, 'R')

session_quali.load()
session_quali.results

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.1.6]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No ca

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points
1,1,M VERSTAPPEN,VER,max_verstappen,Red Bull Racing,3671C6,red_bull,Max,Verstappen,Max Verstappen,https://www.formula1.com/content/dam/fom-websi...,,1.0,1,1.0,NaT,NaT,NaT,0 days 01:33:56.736000,Finished,25.0
11,11,S PEREZ,PER,perez,Red Bull Racing,3671C6,red_bull,Sergio,Perez,Sergio Perez,https://www.formula1.com/content/dam/fom-websi...,,2.0,2,2.0,NaT,NaT,NaT,0 days 00:00:11.987000,Finished,18.0
14,14,F ALONSO,ALO,alonso,Aston Martin,358C75,aston_martin,Fernando,Alonso,Fernando Alonso,https://www.formula1.com/content/dam/fom-websi...,,3.0,3,5.0,NaT,NaT,NaT,0 days 00:00:38.637000,Finished,15.0
55,55,C SAINZ,SAI,sainz,Ferrari,F91536,ferrari,Carlos,Sainz,Carlos Sainz,https://www.formula1.com/content/dam/fom-websi...,,4.0,4,4.0,NaT,NaT,NaT,0 days 00:00:48.052000,Finished,12.0
44,44,L HAMILTON,HAM,hamilton,Mercedes,6CD3BF,mercedes,Lewis,Hamilton,Lewis Hamilton,https://www.formula1.com/content/dam/fom-websi...,,5.0,5,7.0,NaT,NaT,NaT,0 days 00:00:50.977000,Finished,10.0
18,18,L STROLL,STR,stroll,Aston Martin,358C75,aston_martin,Lance,Stroll,Lance Stroll,https://www.formula1.com/content/dam/fom-websi...,,6.0,6,8.0,NaT,NaT,NaT,0 days 00:00:54.502000,Finished,8.0
63,63,G RUSSELL,RUS,russell,Mercedes,6CD3BF,mercedes,George,Russell,George Russell,https://www.formula1.com/content/dam/fom-websi...,,7.0,7,6.0,NaT,NaT,NaT,0 days 00:00:55.873000,Finished,6.0
77,77,V BOTTAS,BOT,bottas,Alfa Romeo,C92D4B,alfa,Valtteri,Bottas,Valtteri Bottas,https://www.formula1.com/content/dam/fom-websi...,,8.0,8,12.0,NaT,NaT,NaT,0 days 00:01:12.647000,Finished,4.0
10,10,P GASLY,GAS,gasly,Alpine,2293D1,alpine,Pierre,Gasly,Pierre Gasly,https://www.formula1.com/content/dam/fom-websi...,,9.0,9,20.0,NaT,NaT,NaT,0 days 00:01:13.753000,Finished,2.0
23,23,A ALBON,ALB,albon,Williams,37BEDD,williams,Alexander,Albon,Alexander Albon,https://www.formula1.com/content/dam/fom-websi...,,10.0,10,15.0,NaT,NaT,NaT,0 days 00:01:29.774000,Finished,1.0


### Explore Laps

In [30]:
lap_data = session_number.laps
print(lap_data)

                      Time Driver DriverNumber                LapTime  \
0   0 days 00:16:58.835000    VER            1                    NaT   
1   0 days 00:18:25.719000    VER            1 0 days 00:01:26.884000   
2   0 days 00:20:23.942000    VER            1 0 days 00:01:58.223000   
3   0 days 00:21:50.430000    VER            1 0 days 00:01:26.488000   
4   0 days 00:23:32.250000    VER            1 0 days 00:01:41.820000   
..                     ...    ...          ...                    ...   
339 0 days 00:26:04.310000    ZHO           24 0 days 00:01:27.432000   
340 0 days 00:27:45.619000    ZHO           24 0 days 00:01:41.309000   
341 0 days 00:31:55.693000    ZHO           24                    NaT   
342 0 days 00:33:24.257000    ZHO           24 0 days 00:01:28.564000   
343 0 days 00:35:20.439000    ZHO           24 0 days 00:01:56.182000   

     LapNumber  Stint             PitOutTime              PitInTime  \
0          1.0    1.0 0 days 00:15:18.665000        

In [35]:
lap_data.pick_team('McLaren').pick_fastest()

Time                      0 days 01:07:22.657000
Driver                                       PIA
DriverNumber                                  81
LapTime                   0 days 00:01:24.540000
LapNumber                                   19.0
Stint                                        5.0
PitOutTime                                   NaT
PitInTime                                    NaT
Sector1Time               0 days 00:00:31.266000
Sector2Time               0 days 00:00:28.508000
Sector3Time               0 days 00:00:24.766000
Sector1SessionTime        0 days 01:06:29.383000
Sector2SessionTime        0 days 01:06:57.891000
Sector3SessionTime        0 days 01:07:22.657000
SpeedI1                                    245.0
SpeedI2                                    289.0
SpeedFL                                    283.0
SpeedST                                    314.0
IsPersonalBest                              True
Compound                                    SOFT
TyreLife            

In [36]:
lap_data.pick_drivers(['PIA', 'NOR'])

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
107,0 days 00:19:05.790000,PIA,81,NaT,1.0,1.0,0 days 00:17:00.616000,NaT,NaT,0 days 00:00:36.155000,0 days 00:00:29.249000,NaT,0 days 00:18:36.552000,0 days 00:19:05.973000,208.0,267.0,282.0,33.0,False,SOFT,1.0,True,McLaren,0 days 00:17:00.616000,2023-10-06 17:02:00.621,1,NaN,False,,False,False
108,0 days 00:20:32.513000,PIA,81,0 days 00:01:26.723000,2.0,1.0,NaT,NaT,0 days 00:00:32.186000,0 days 00:00:29.260000,0 days 00:00:25.277000,0 days 00:19:37.976000,0 days 00:20:07.236000,0 days 00:20:32.513000,238.0,287.0,281.0,316.0,True,SOFT,2.0,True,McLaren,0 days 00:19:05.790000,2023-10-06 17:04:05.795,1,NaN,False,,False,True
109,0 days 00:22:27.374000,PIA,81,0 days 00:01:54.861000,3.0,1.0,NaT,NaT,0 days 00:00:44.265000,0 days 00:00:39.192000,0 days 00:00:31.404000,0 days 00:21:16.778000,0 days 00:21:55.970000,0 days 00:22:27.374000,98.0,254.0,284.0,294.0,False,SOFT,3.0,True,McLaren,0 days 00:20:32.513000,2023-10-06 17:05:32.518,1,NaN,False,,False,True
110,0 days 00:24:00.113000,PIA,81,0 days 00:01:32.739000,4.0,1.0,NaT,0 days 00:23:55.921000,0 days 00:00:31.802000,0 days 00:00:29.589000,0 days 00:00:31.348000,0 days 00:22:59.176000,0 days 00:23:28.765000,0 days 00:24:00.113000,242.0,287.0,NaN,315.0,False,SOFT,4.0,True,McLaren,0 days 00:22:27.374000,2023-10-06 17:07:27.379,1,NaN,False,,False,False
111,0 days 00:28:38.105000,PIA,81,NaT,5.0,2.0,0 days 00:26:42.340000,NaT,NaT,0 days 00:00:33.252000,0 days 00:00:43.313000,NaT,0 days 00:27:54.792000,0 days 00:28:38.276000,209.0,258.0,285.0,75.0,False,SOFT,1.0,True,McLaren,0 days 00:24:00.113000,2023-10-06 17:09:00.118,1,NaN,False,,False,False
112,0 days 00:30:03.874000,PIA,81,0 days 00:01:25.769000,6.0,2.0,NaT,NaT,0 days 00:00:31.661000,0 days 00:00:29.074000,0 days 00:00:25.034000,0 days 00:29:09.766000,0 days 00:29:38.840000,0 days 00:30:03.874000,243.0,288.0,283.0,316.0,True,SOFT,2.0,True,McLaren,0 days 00:28:38.105000,2023-10-06 17:13:38.110,1,NaN,False,,False,True
113,0 days 00:32:04.986000,PIA,81,0 days 00:02:01.112000,7.0,2.0,NaT,NaT,0 days 00:00:43.162000,0 days 00:00:39.142000,0 days 00:00:38.808000,0 days 00:30:47.036000,0 days 00:31:26.178000,0 days 00:32:04.986000,172.0,251.0,286.0,256.0,False,SOFT,3.0,True,McLaren,0 days 00:30:03.874000,2023-10-06 17:15:03.879,1,NaN,False,,False,True
114,0 days 00:33:30.252000,PIA,81,0 days 00:01:25.266000,8.0,2.0,NaT,NaT,0 days 00:00:31.399000,0 days 00:00:28.992000,0 days 00:00:24.875000,0 days 00:32:36.385000,0 days 00:33:05.377000,0 days 00:33:30.252000,246.0,288.0,286.0,320.0,True,SOFT,4.0,True,McLaren,0 days 00:32:04.986000,2023-10-06 17:17:04.991,1,NaN,False,,False,True
115,0 days 00:35:25.592000,PIA,81,0 days 00:01:55.340000,9.0,2.0,NaT,0 days 00:35:21.425000,0 days 00:00:38.562000,0 days 00:00:36.472000,0 days 00:00:40.306000,0 days 00:34:08.814000,0 days 00:34:45.286000,0 days 00:35:25.592000,167.0,246.0,NaN,301.0,False,SOFT,5.0,True,McLaren,0 days 00:33:30.252000,2023-10-06 17:18:30.257,1,NaN,False,,False,False
116,0 days 00:42:28.939000,PIA,81,NaT,10.0,3.0,0 days 00:40:40.748000,NaT,NaT,0 days 00:00:34.309000,0 days 00:00:35.851000,NaT,0 days 00:41:53.090000,0 days 00:42:28.949000,201.0,237.0,283.0,121.0,False,SOFT,6.0,False,McLaren,0 days 00:35:25.592000,2023-10-06 17:20:25.597,1,NaN,False,,False,False


### Compare DriverId field to names

In [37]:
df_all = pd.DataFrame()

list_years = list(range(2015, 2023 + 1))

for year in list_years:
    print(f"Pulling data for {year}")

    schedule = ff1.get_event_schedule(year)
    rounds = schedule.RoundNumber.to_list()

    for round in rounds:
        print(f"Pulling data for round {round}")

        session = ff1.get_session(year, round, "R")
        session.load()
        df_race = session.results[["DriverId", "LastName", "FirstName"]]

        df_all.append(df_race, ignore_index=True)

logger      WARNING 	Failed to load schedule from Ergast API backend!


Pulling data for 2015


ValueError: Failed to load any schedule data.